This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import json
from collections import defaultdict
from datetime import datetime

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get("https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json?api_key=" + API_KEY)

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
# r.json()

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
# TASK 1
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)

In [7]:
# TASK 2
json_data = r.json()

In [8]:
d = json_data['dataset']['data']
headers = json_data['dataset']['column_names']

In [9]:
data = defaultdict(datetime)
for row in d:
    row_dict = {}
    i = 0
    for col in headers:
        row_dict[col] = row[i]
        i += 1
    date = datetime.strptime(row_dict['Date'], '%Y-%m-%d')
    data[date] = row_dict
print(data)

st Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}, datetime.datetime(2017, 4, 13, 0, 0): {'Date': '2017-04-13', 'Open': 42.06, 'High': 42.48, 'Low': 41.99, 'Close': 42.2, 'Change': None, 'Traded Volume': 88416.0, 'Turnover': 3734717.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}, datetime.datetime(2017, 4, 12, 0, 0): {'Date': '2017-04-12', 'Open': 42.02, 'High': 42.45, 'Low': 41.84, 'Close': 42.2, 'Change': None, 'Traded Volume': 158278.0, 'Turnover': 6672547.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}, datetime.datetime(2017, 4, 11, 0, 0): {'Date': '2017-04-11', 'Open': 41.62, 'High': 42.03, 'Low': 41.53, 'Close': 41.75, 'Change': None, 'Traded Volume': 107817.0, 'Turnover': 4501109.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}, datetime.datetime(2017, 4, 10, 0, 0): {'Date': '2017-04-10', 'Open': 41.46, 'High': 41.68, 'Low': 41.31, 'Clo

In [10]:
# There is some missing data we will have to deal with
dates_to_skip = []
for date in data:
    if data[date]['Open'] == None:
        dates_to_skip.append(date)
        print(date)

2017-05-01 00:00:00
2017-04-17 00:00:00
2017-04-14 00:00:00


In [11]:
# TASK 3
max_open = 0
max_open_date = datetime(2017, 1, 1)
min_open = 10000000
min_open_date = datetime(2017, 1, 1)
for key in data:
    if key in dates_to_skip:
        pass
    elif data[key]['Open'] == 0:
        pass
    elif data[key]['Open'] > max_open:
        max_open = data[key]['Open']
        max_open_date = key
    elif data[key]['Open'] < min_open:
        min_open = data[key]['Open']
        min_open_date = key
print('Highest opening price: $', max_open, ' on ', str(max_open_date))
print('Lowest opening price: $', min_open, ' on ', str(min_open_date))

Highest opening price: $ 53.11  on  2017-12-14 00:00:00
Lowest opening price: $ 34.0  on  2017-01-24 00:00:00


In [12]:
dates_to_skip = []
for date in data:
    if data[date]['High'] == None or data[date]['Low'] == None:
        dates_to_skip.append(date)
        print(date)

In [13]:
# TASK 4: What was the largest change in any one day (based on High and Low price)?
max_change = 0
max_change_date = datetime(2017, 1, 1)
for key in data:
    if key in dates_to_skip:
        pass
    else:
        change = abs(data[key]['High'] - data[key]['Low'])
        if change > max_change:
            max_change = change
            max_change_date = key
print('Biggest change: $', round(max_change, 2), 'on', max_change_date)

Biggest change: $ 2.81 on 2017-05-11 00:00:00


In [14]:
dates_to_skip = []
for date in data:
    if data[date]['Close'] == None:
        dates_to_skip.append(date)
        print(date)

In [15]:
import numpy as np

In [16]:
# TASK 5: What was the largest change between any two days (based on Closing Price)?
closing = []
dates = []
for key in data:
    if key in dates_to_skip:
        pass
    else:
        closing.append(data[key]['Close'])
        dates.append(key)

# List slicing
cls1 = np.array(closing[:-1])
cls2 = np.array(closing[1:])
dt1 = dates[:-1]
dt2 = dates[1:]

# Element-wise difference
cls_diff = abs(cls2 - cls1)

# Find the max change, the index of that will be analogous to cls1, index in cls2 will be 1 more
max_change = max(cls_diff)
max_change_index = np.argmax(cls_diff)

day1 = dt1[max_change_index]
day2 = dt2[max_change_index + 1]

print('Biggest change: $', round(max_change, 2), 'from', day1, 'to', day2)

Biggest change: $ 2.56 from 2017-08-09 00:00:00 to 2017-08-07 00:00:00


In [17]:
# TASK 6: What was the average daily trading volume during this year?
sum_vol = 0
for key in data:
    sum_vol += data[key]['Traded Volume']
print('Average daily trading volume:', round(sum_vol/len(data), 3))

Average daily trading volume: 89124.337


In [19]:
# TASK 7: (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
# Make an array of the trading volumnes
volumes = [data[key]['Traded Volume'] for key in data]
# Take the np.median
median = np.median(volumes)
print("Median trading volume:",median)

Median trading volume: 76286.0
